## simcat Analysis demo

In [1]:
# conda install multicore-tsne -c conda-forge
# conda install toyplot -c eaton-lab

In [2]:
import simcat
import toytree
import toyplot
import toyplot.svg
import numpy as np
from MulticoreTSNE import MulticoreTSNE as TSNE

### Load Database results

#### Set labels to unordered labels

In [9]:
ml1 = simcat.Analysis(
    name="test-111",
    workdir="./databases",
    ulabel=True,
)

subset data to: 1056 tests
Dataset: test-111
loaded counts matrix: (1056, 5, 16, 16)
scaled integers to floats by max count
reshaped into X: (1056, 1280)
loaded labels DataFrame: (1056, 6)
subset as y: (1056,)
added 5 features from abba-baba
split train/test data: (707, 1285)/(349, 1285)


#### Mask (set label to NaN) low migration and/or sister edges

In [19]:
ml2 = simcat.Analysis(
    name="test-111",
    workdir="./databases",
    mask_admixture_min=0.02,
    mask_sisters=True,
    ulabel=True,
)

subset data to: 1056 tests
Dataset: test-111
loaded counts matrix: (1056, 5, 16, 16)
scaled integers to floats by max count
reshaped into X: (1056, 1280)
loaded labels DataFrame: (1056, 6)
subset as y: (1056,)
added 5 features from abba-baba
split train/test data: (707, 1285)/(349, 1285)


#### Exclude low migration and/or sisters

In [20]:
ml3 = simcat.Analysis(
    name="test-111",
    workdir="./databases",
    exclude_admixture_min=0.05,
    exclude_sisters=True,
)

subset data to: 648 tests
Dataset: test-111
loaded counts matrix: (648, 5, 16, 16)
scaled integers to floats by max count
reshaped into X: (648, 1280)
loaded labels DataFrame: (648, 6)
subset as y: (648,)
added 5 features from abba-baba
split train/test data: (434, 1285)/(214, 1285)


#### Other setups

In [21]:
ml4 = simcat.Analysis(
    name="test-111",
    workdir="./databases",
    exclude_admixture_min=0.1,
    exclude_sisters=True,
    ulabel=True,
)

subset data to: 576 tests
Dataset: test-111
loaded counts matrix: (576, 5, 16, 16)
scaled integers to floats by max count
reshaped into X: (576, 1280)
loaded labels DataFrame: (576, 6)
subset as y: (576,)
added 5 features from abba-baba
split train/test data: (385, 1285)/(191, 1285)


### Fit Classifier
This will fit an extra trees classifier to try to classify the placement of edges on the tree. It is a crude yes/no answer, and we may want to also report incorrect edges in terms of their distance off... or something. 

In [22]:
ml1.train_model()

training ExtraTrees model...
model score on training set: 1.000
model score on test/validation set: 0.751


In [23]:
ml2.train_model()

training ExtraTrees model...
model score on training set: 1.000
model score on test/validation set: 0.722


In [24]:
ml3.train_model()

training ExtraTrees model...
model score on training set: 1.000
model score on test/validation set: 0.860


In [25]:
ml4.train_model()

training ExtraTrees model...
model score on training set: 1.000
model score on test/validation set: 0.979


### Visualize features

In [94]:
from sklearn.decomposition import PCA

In [95]:
pX = PCA().fit_transform(ml.X)
pX.shape

(1056, 1056)

In [96]:
tsne = TSNE(
    init="random",
    perplexity=50, 
    n_iter=20000, 
    n_jobs=8,
    random_state=123,
)

In [97]:
embedding = tsne.fit_transform(pX)#ml.X)

In [98]:
# greys = toyplot.color.brewer.palette("Greys", 3)
# toyplot.color.to_css(greys[1]), toyplot.color.to_css(greys[0])

In [99]:
# spec = toyplot.color.brewer.palette("Spectral", count=11)
# spec

In [103]:
colors = toyplot.color.brewer.palette("Paired")
colors

In [110]:
ml.df.ulabel[ml.df.sisters == 0]

3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
       ... 
1048    0,3
1049    0,3
1050    0,2
1051    0,2
1052    0,2
Name: ulabel, Length: 792, dtype: object

In [112]:
cdict = {
    "NaN": {
        "fill": 'rgba(74.1%,74.1%,74.1%,1.000)', 
        'stroke': 'none',
    },
}
labels = set(ml.df.ulabel[ml.df.sisters == 0])
labels = labels - {"NaN"}
for label, color in zip(labels, colors):
    
    # set ->
    cdict[label] = {
        "stroke": "none",
        "fill": toyplot.color.to_css(color),
    }
    
    # set <-
    alt = "{1},{0}".format(*label.split(","))
    cdict[alt] = {
        "stroke": toyplot.color.to_css(color),
        "stroke-width": 1.5,
        "fill": 'rgba(100%,100%,100%,1.000)',
    }

In [118]:
markers = []
for idx in ml.df.index:
    if ml.df.sisters[idx]:
        mark = toyplot.marker.create(
            shape="o",
            size= 3, #+ (ml.df.aprop[idx] / ml.df.aprop.max()) * 10,# * 35,
            mstyle=cdict["NaN"])
    
    else:
        mark = toyplot.marker.create(
            shape='o',
            size= 3 + (ml.df.aprop[idx] / ml.df.aprop.max()) * 10,# * 35,
            mstyle=cdict[ml.df.label[idx]],
        )
    markers.append(mark)

In [121]:
canvas = toyplot.Canvas(width=800, height=400,)
ax0 = canvas.cartesian(bounds=(50, 350, 70, 330))
ax1 = canvas.cartesian(bounds=(450, 700, 100, 300))

ax0.scatterplot(
    embedding[:, 0],
    embedding[:, 1],
    marker=markers,
    title=ml.df.label,
);

tre = toytree.tree(ml.tree)
tre.draw(ts='c', axes=ax1, node_sizes=18, node_colors="lightgrey");
ax1.y.show = False
ax0.x.label.text = "t-SNE axis 1"
ax0.y.label.text = "t-SNE axis 2"

# save image
#toyplot.svg.render(canvas, "./tsne-imb5-snps10K-11tests-withsis2.svg")

<svg class="toyplot-canvas-Canvas" height="400.0px" id="ta4cf9d95827046ec8c5fbf15dbdddf7d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 400.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0

In [193]:
toyplot.scatterplot(
    embedding[:, 0],
    embedding[:, 1],
    width=400, height=400,
    marker=markers,
    title=ml.df.label,
);
toytree.tree(ml.tree).draw(ts='c');

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t60cf9c51ab314916bca8bd9d3f1ae783" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 3,4 3,4 3,4 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 2,4 2,4 2,4 2,3 2,3 2,3 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 -20 -10 0 10 20 -20 -10 0 10

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t0fd447250de649b289c537774a4a5800" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 1250000
support: 100
height: 0 0 idx: 1
name: r0
dist: 1250000
support: 100
height: 0 1 idx: 2
name: r2
dist: 2500000
support: 100
height: 0 2 idx: 3
name: r3
dist: 3750000
support: 100
height: 0 3 idx: 4
name: r4
dist: 5000000
support: 100
height: 0 4 idx: 5
name: 5
dist: 1250000
support: 100
height: 1250000 5 idx: 6
name: 6
dist: 1250000
support: 100
height: 2500000 6 idx: 7
name: 7
dist: 1250000
support: 100
height: 3750000 7 idx: 8
name: 8
dist: 1
support: 100
height: 5000000 8 0 2500000 5000000

In [140]:
toyplot.scatterplot(
    embedding[:, 0],
    embedding[:, 1],
    width=400, height=400,
    marker=markers,
    title=ml.df.label,
);
toytree.tree(ml.tree).draw(ts='c');

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t74bf586272dd4895bd2b827ca1f96758" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 4,3 4,3 4,6 4,6 4,6 4,2 4,2 4,2 4,5 4,5 4,5 4,1 4,1 4,1 4,0 4,0 4,0 7,4 7,4 7,4 3,4 3,4 3,4 3,6 3,6 3,6 3,2 3,2 3,2 3,5 3,5 3,5 3,1 3,1 3,1 3,0 3,0 3,0 6,4 6,4 6,4 6,3 6,3 6,3 2,4 2,4 2,4 2,3 2,3 2,3 2,5 2,5 2,5 2,1 2,1 2,1 2,0 2,0 2,0 5,4 5,4 5,4 5,3 5,3 5,3 5,2 5,2 5,2 1,4 1,4 1,4 1,3 1,3 1,3 1,2 1,2 1,2 1,0 1,0 1,0 0,4 0,4 0,4 0,3 0,3 0,3 0,2 0,2 0,2 0,1 0,1 0,1 4,7 4,7 4,7 4,3 <g class="toyplot-Datum" style="fill:rgb(89%,10.2%,11%);fill-opacity:1.0;opacity:1.0;stroke:rgb(0%,0%,0%);stroke-opacity:1.

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t797124e5549447469477b1a6e138b113" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 1250000
support: 100
height: 0 0 idx: 1
name: r0
dist: 1250000
support: 100
height: 0 1 idx: 2
name: r2
dist: 2500000
support: 100
height: 0 2 idx: 3
name: r3
dist: 3750000
support: 100
height: 0 3 idx: 4
name: r4
dist: 5000000
support: 100
height: 0 4 idx: 5
name: 5
dist: 1250000
support: 100
height: 1250000 5 idx: 6
name: 6
dist: 1250000
support: 100
height: 2500000 6 idx: 7
name: 7
dist: 1250000
support: 100
height: 3750000 7 idx: 8
name: 8
dist: 1
support: 100
height: 5000000 8 0 2500000 5000000